# Instacart Product Recommendation

In [1]:
import pandas as pd
import numpy as np

In [11]:
products = pd.read_csv('../../data/01_raw/instacart_2017_05_01/products.csv')
aisles = pd.read_csv('../../data/01_raw/instacart_2017_05_01/aisles.csv')
departments = pd.read_csv('../../data/01_raw/instacart_2017_05_01/departments.csv')

order_products__prior = pd.read_csv('../../data/01_raw/instacart_2017_05_01/order_products__prior.csv')
order_products__train = pd.read_csv('../../data/01_raw/instacart_2017_05_01/order_products__train.csv')
order_test = pd.read_csv('../../data/01_raw/instacart_2017_05_01/orders.csv')

In [10]:
len(order_products__prior)

32434489

In [6]:
prod_ailes = products.merge(aisles, 
              how='outer', 
              on='aisle_id', 
               suffixes=('_x', '_y')
              )

In [7]:
product_dataset = prod_ailes.merge(departments, 
                how='outer', 
                on='department_id')

In [12]:
product_dataset.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


Let's make a dataset that allows us to see what items go with what orders 

In [26]:
specific_orders = order_products__prior.merge(product_dataset, 
                how='left', 
                on='product_id')

Now let's add user id information

In [36]:
baskets = specific_orders.merge(order_test, 
                     how='left', 
                     on='order_id')

Great, now let's see what we have! 

In [39]:
baskets.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry,202279,prior,3,5,9,8.0


Our data consists of over 200K unique users making orders and over 3 million unique orders. 

In [46]:
baskets.user_id.nunique()

206209

In [47]:
baskets.order_id.nunique()

3214874

Let's take a look at the types of products we have

In [49]:
# 50k unique products
baskets.product_name.nunique()

49677

How many times does each user appear in the dataset? 

In [53]:
baskets.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'aisle_id', 'department_id', 'aisle', 'department',
       'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')

As we can see, we have over 200K different people making purchases at different frequencies and different amounts. 

In [57]:
baskets.groupby(['user_id', 'order_id']).count()

product_id  add_to_cart_order  reordered  product_name  \
user_id order_id                                                           
1       431534             8                  8          8             8   
        473747             5                  5          5             5   
        550135             5                  5          5             5   
        2254736            5                  5          5             5   
        2295261            6                  6          6             6   
...                      ...                ...        ...           ...   
206209  2307371            3                  3          3             3   
        2558525            3                  3          3             3   
        2977660            9                  9          9             9   
        3154581           13                 13         13            13   
        3186442            2                  2          2             2   

                  aisle_id  department_id  aisle  department  eval_set  \
user_id order_id                                                         
1       431534           8              8      8           8         8   
        473747           5              5      5           5         5   
        550135           5              5      5           5         5   
        2254736          5              5      5           5         5   
        2295261          6              6      6           6         6   
...                    ...            ...    ...         ...       ...   
206209  2307371          3              3      3           3         3   
        2558525          3              3      3           3         3   
        2977660          9              9      9           9         9   
        3154581         13             13     13          13        13   
        3186442          2              2      2           2         2   

                  order_number  order_dow  order_hour_of_day  \
user_id order_id                                               
1       431534               8          8                  8   
        473747               5          5                  5   
        550135               5          5                  5   
        2254736              5          5                  5   
        2295261              6          6                  6   
...                        ...        ...                ...   
206209  2307371              3          3                  3   
        2558525              3          3                  3   
        2977660              9          9                  9   
        3154581             13         13                 13   
        3186442              2          2                  2   

                  days_since_prior_order  
user_id order_id                          
1       431534                         8  
        473747                         5  
        550135                         5  
        2254736                        5  
        2295261                        6  
...                                  ...  
206209  2307371                        3  
        2558525                        3  
        2977660                        9  
        3154581                        0  
        3186442                        2  

[3214874 rows x 13 columns]

In [51]:
baskets.department.unique()

array(['dairy eggs', 'produce', 'pantry', 'meat seafood', 'bakery',
       'personal care', 'snacks', 'breakfast', 'beverages', 'deli',
       'household', 'international', 'dry goods pasta', 'frozen',
       'canned goods', 'babies', 'pets', 'alcohol', 'bulk', 'missing',
       'other'], dtype=object)

https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6